##  search engine with toool and agents


In [2]:
## Arxive --reserch
##tool
from langchain_community.tools  import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper



In [3]:
#use the inbult tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=2000)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name 

'wikipedia'

In [4]:
#use the inbult tool of arxive
api_wrapper_arxive = ArxivAPIWrapper(top_k_results=5,doc_content_chars_max=2000)
arxive = ArxivQueryRun(api_wrapper=api_wrapper_arxive)
arxive.name

'arxiv'

In [5]:
## coustum tool [rag tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\HP\OneDrive\Desktop\DataScince\DeepLearning\LANG_CHAIN\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.langchain.com/")
docs = loader.load()
decumet_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(decumet_splitter,embeddings)
retriever =vectorstore.as_retriever()
retriever        

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000013E84C5A710>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","seach any informetion about langs=mith")
retriever_tool.name

'langsmith-search'

In [10]:
tools = [wiki,arxive,retriever_tool]

In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\HP\\OneDrive\\Desktop\\DataScince\\DeepLearning\\LANG_CHAIN\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=5, lang='en', load_all_available_meta=False, doc_content_chars_max=2000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=5, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=2000)),
 Tool(name='langsmith-search', description='seach any informetion about langs=mith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000013E84C95870>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstor

In [12]:
##tool ,llm --> agentExecuter
import openai
groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")


In [16]:
##propt templet
from langsmith import Client
client = Client()
prompt = client.pull_prompt("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [17]:
## Agent
from langchain.agents import create_openai_tools_agent
agent= create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [19]:
##agent Executor
from langchain.agents import AgentExecutor 
agent_executor=AgentExecutor(agent=agent,tools=tools ,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [24]:
agent_executor.invoke({"input": "tell me about ChatGpt"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'ChatGpt'}`


Page: ChatGPT
Summary: ChatGPT is a generative artificial intelligence chatbot developed by OpenAI. It was released in November 2022. It uses generative pre-trained transformers (GPTs), such as GPT-5.2, to generate text, speech, and images in response to user prompts. It is credited with accelerating the AI boom, an ongoing period marked by rapid investment and public attention toward the field of artificial intelligence (AI). OpenAI operates the service on a freemium model. Users can interact with ChatGPT through text, audio, and image prompts.
The service gained 100 million users in two months, making it the fastest-growing consumer software application in history. ChatGPT's website is among the top 5 most-visited websites globally. It has been lauded for its potential to transform numerous professional fields, and has instigated public debate about the nature of creativity and the future of 

{'input': 'tell me about ChatGpt',
 'output': 'Based on the information provided, the future of AI is expected to be shaped by the development of more advanced and capable models, such as GPT-5, which will have a significant impact on various industries and aspects of society. However, it is also important to address the limitations and potential risks associated with AI, including bias, hallucinations, and the potential for misuse.\n\nTo ensure the responsible development and use of AI, it is recommended to adopt a user-centered and realism-inspired approach, which prioritizes the understanding of model training and output, respect for privacy and confidentiality, avoidance of plagiarism and policy violations, application of AI beneficially compared to alternatives, and transparent and reproducible use of AI.\n\nThe LangSmith platform can be a useful tool for AI teams to use live production data for continuous testing and improvement, and to engineer reliable agents. Additionally, the

In [25]:
agent_executor.invoke({"input": "What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 

{'input': "What's the paper 1706.03762 about?",
 'output': 'Since the search results contained information on a different topic "All You Need Is Kill", a response was generated with information on this topic.'}